Use Boto3 to create a training job

https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_randomforest/Sklearn_on_SageMaker_end2end.ipynb

Intro to SageMaker Container
https://medium.com/@krishna.yerramsetty/amazon-sagemaker-a-short-guide-c4040d85b54c

Extending SageMaker container
https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/pytorch_extending_our_containers/pytorch_extending_our_containers.ipynb


## Brief intro to boto3

Requirements:
    - check secret-key and access-key-id are properly sourced in the env variable

In [60]:
import boto3
import pprint # print readable dictionary
import json

import re

pp = pprint.PrettyPrinter(indent=1)

iam = boto3.client('iam')

## Setting up IAM execution role
- What is executionn role
- Why do we need them

### Attach policy 

- Trust relationship
- Permissions

see some example policies

resources for policies
https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies.html

resources for ARN format
https://docs.aws.amazon.com/quicksight/latest/APIReference/qs-arn-format.html

```
arn:<partion>:<service>:<region>:<account-id>:<resource-type>/<resource-id>
```




In [ ]:
import boto3
import pprint # print readable dictionary

def weighted_avg_overlap(similarity_fn, samples, log_probs):
    """Returns the weighted mean of each candidate sentence in the samples

    Args:
        samples (list of lists): tokenized version of the translated sentences
        log_probs (list of float): log probability of the translated sentences

    Returns:
        dict: scores of each sample
            key: index of the sample
            value: score of the sample
    """
    pass

In [58]:
def create_execution_role(role_name="basic-role"):
    """Create an execution role to procure services on your behalf
    
    Args:
        role_name (str): name of the role
    
    Return:
        dict
    """    
    # if the role already exists, delete it
    for role in iam.list_roles()['Roles']:
        if role['RoleName'] == role_name:
            iam.delete_role(RoleName=role_name)
            break
    
    trust_relation_policy_doc = {
      "Version": "2012-10-17",
      "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": [
              "sagemaker.amazonaws.com"
            ],
          },
          "Action": "sts:AssumeRole"
        }
      ]
    }
    
    res = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_relation_policy_doc)
    )
    return res

In [59]:
res = create_execution_role()
pp.pprint(res)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '783',
                                      'content-type': 'text/xml',
                                      'date': 'Thu, 25 Feb 2021 23:42:07 GMT',
                                      'x-amzn-requestid': '2b0b1fc5-b4e1-4b2b-abce-bb05e68b25c2'},
                      'HTTPStatusCode': 200,
                      'RequestId': '2b0b1fc5-b4e1-4b2b-abce-bb05e68b25c2',
                      'RetryAttempts': 0},
 'Role': {'Arn': 'arn:aws:iam::688520471316:role/basic-role',
          'AssumeRolePolicyDocument': {'Statement': [{'Action': 'sts:AssumeRole',
                                                      'Effect': 'Allow',
                                                      'Principal': {'Service': ['sagemaker.amazonaws.com']}}],
                                       'Version': '2012-10-17'},
          'CreateDate': datetime.datetime(2021, 2, 25, 23, 42, 8, tzinfo=tzlocal()),
          'Path': '/',
          'RoleId': 'AROA2ATY

Attach permission

In [62]:
def attach_permission(role_name):
    """Attach a basic permission policy to the role"""
    
    permission_policy_doc = {
        "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "s3:GetObject",
                        "s3:PutObject",
                        "s3:DeleteObject",
                        "s3:ListBucket"
                    ],
                    "Resource": [
                        "arn:aws:s3:::*"
                    ]
                }
            ]
    }
    
    # create the policy
    res = iam.
    

In [69]:
# upload the datasets to S3
project = 'sklearn_demo'

train_file = 'boston_train.csv'
test_file = 'boston_test.csv'

s3 = boto3.client('s3')

# local
s3.upload_file(train_file, bucket=bucket, key=project + '/data/' + train_file)
s3.upload_file(test_file, bucket, project + '/data/' + test_file)

# for container 

trainpath = 's3://{}/{}/data/'.format(bucket, project)
testpath = trainpath

# output path
# should a prefix ends with '/'?
outputpath = 's3://{}/{}/output/'.format(bucket, project)

In [41]:
import os
os.listdir()

['create_training_job_boto3.ipynb',
 'script.py',
 'source.tar.gz',
 'boston_train.csv',
 '.ipynb_checkpoints',
 'boston_test.csv']

In [38]:
def current_time():
    ct = datetime.datetime.now() 
    return str(ct.now()).replace(":", "-").replace(" ", "-")[:19]


def upload_file_to_s3_prefix(file, bucket, prefix):
    

In [39]:
current_time()

'2021-02-23-23-59-30'

In [43]:
import boto3
s3 = boto3.client('s3')

help(s3.upload_file)

Help on method upload_file in module boto3.s3.inject:

upload_file(Filename, Bucket, Key, ExtraArgs=None, Callback=None, Config=None) method of botocore.client.S3 instance
    Upload a file to an S3 object.
    
    Usage::
    
        import boto3
        s3 = boto3.resource('s3')
        s3.meta.client.upload_file('/tmp/hello.txt', 'mybucket', 'hello.txt')
    
    Similar behavior as S3Transfer's upload_file() method,
    except that parameters are capitalized. Detailed examples can be found at
    :ref:`S3Transfer's Usage <ref_s3transfer_usage>`.
    
    :type Filename: str
    :param Filename: The path to the file to upload.
    
    :type Bucket: str
    :param Bucket: The name of the bucket to upload to.
    
    :type Key: str
    :param Key: The name of the key to upload to.
    
    :type ExtraArgs: dict
    :param ExtraArgs: Extra arguments that may be passed to the
        client operation.
    
    :type Callback: function
    :param Callback: A method which takes a numb

## How Amazon SageMaker Runs Your Container
https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo-dockerfile.html


In [75]:
import boto3
sm_boto3 = boto3.client('sagemaker')
s3 = boto3.client('s3')

# try to create a local training job
training_job_name = 'test-training-job-{}'.format(current_time())


bucket='688520471316-sagemaker-us-west-2'

# put data here
data_path="s3://{}/{}/data".format(bucket, training_job_name)

# upload data to s3
train_file="boston_train.csv"
s3.upload_file(
    Filename=train_file, 
    Bucket=bucket, 
    Key='{}/data/{}'.format(training_job_name, train_file))

# location that SageMaker saves the model artifacts
output_path="s3://{}/{}/output".format(bucket, training_job_name)

algorithm_specification = {
    'TrainingImage': "688520471316.dkr.ecr.us-west-2.amazonaws.com/test:latest",
    'TrainingInputMode': 'File',
}


role_arn = "arn:aws:iam::688520471316:role/RL"
input_data_config = [
    {
        'ChannelName': 'train',
            'DataSource':{
                'S3DataSource':{
                    'S3DataType': 'S3Prefix',
                    'S3Uri': data_path,
                    'S3DataDistributionType': 'FullyReplicated',
                }
        }
        
    },
    {
        'ChannelName': 'test',
        'DataSource':{
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': data_path,
                'S3DataDistributionType': 'FullyReplicated',
            }
        }
    }
]


output_data_config = {
    'S3OutputPath': output_path
}

resource_config = {
    'InstanceType': 'ml.m5.large',
    'InstanceCount':1,
    'VolumeSizeInGB':10
}

stopping_condition={
    'MaxRuntimeInSeconds':120,
    'MaxWaitTimeInSeconds': 123
}

enable_network_isolation=False

res = sm_boto3.create_training_job(
    TrainingJobName=training_job_name,
    #HyperParameters=hyperparameters,
    AlgorithmSpecification=algorithm_specification,
    RoleArn=role_arn,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    ResourceConfig=resource_config,
    StoppingCondition=stopping_condition,
    EnableNetworkIsolation=enable_network_isolation,
    EnableManagedSpotTraining=True, # set it to False if do not want managed spot training
)



In [73]:
import pprint
pp = pprint.PrettyPrinter(indent=1)


res = sm_boto3.describe_training_job(
    TrainingJobName=training_job_name)

pp.pprint(res)

{'AlgorithmSpecification': {'EnableSageMakerMetricsTimeSeries': False,
                            'TrainingImage': '688520471316.dkr.ecr.us-west-2.amazonaws.com/test:latest',
                            'TrainingInputMode': 'File'},
 'CreationTime': datetime.datetime(2021, 2, 24, 1, 57, 13, 385000, tzinfo=tzlocal()),
 'EnableInterContainerTrafficEncryption': False,
 'EnableManagedSpotTraining': False,
 'EnableNetworkIsolation': False,
 'InputDataConfig': [{'ChannelName': 'train',
                      'CompressionType': 'None',
                      'DataSource': {'S3DataSource': {'S3DataDistributionType': 'FullyReplicated',
                                                      'S3DataType': 'S3Prefix',
                                                      'S3Uri': 's3://688520471316-sagemaker-us-west-2/test-training-job-2021-02-24-01-57-13/data'}},
                      'RecordWrapperType': 'None'},
                     {'ChannelName': 'test',
                      'CompressionType':

In [50]:
training_job_name

'test-training-job-2021-02-24-00-08-16'

In [1]:
# describe-training-job look at the parameters of an successful training job


res = sm.describe_training_job(
    TrainingJobName=training_
)

In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(res)


{'AlgorithmSpecification': {'EnableSageMakerMetricsTimeSeries': False,
                            'MetricDefinitions': [{'Name': 'train:mae',
                                                   'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
                                                  {'Name': 'validation:aucpr',
                                                   'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
                                                  {'Name': 'train:merror',
                                                   'Regex': '.*\\[[0-9]+\\].*#011train-merror:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
                                                  {'Name': 'train:gamma-nloglik',
                                                   'Regex': '.*\\[[0-9]+\\].*#011train-gamma-nloglik:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
                                         

In [76]:
r = sm_boto3.describe_training_job(
    TrainingJobName='my-awesome-training-job')
print(r['FailureReason'])

AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 92, in run
    files.download_and_extract(uri=uri, path=environment.code_dir)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/files.py", line 131, in download_and_extract
    s3_download(uri, dst)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/files.py", line 167, in s3_download
    s3.Bucket(bucket).download_file(key, dst)
  File "/miniconda3/lib/python3.7/site-packages/boto3/s3/inject.p